## # FastText 한국어 사전학습 임베딩 cc.ko.100.bin 사용


In [1]:
!pip install konlpy
!pip install wandb
!pip install fasttext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313499 sha256=65fd8a6fa6c8d10ef7b328a71ca1c2af5baabfd61b3c64241826b71512db4b91
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [2]:
import os
import re
import pandas as pd
import numpy as np
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
from konlpy.tag import Okt
import fasttext
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.metrics import F1Score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Input

sys.path.append('/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13/notebooks/jiwoong') # 모듈경로 추가
from utils.preprocessing import *
from utils.models import *

import wandb
from wandb.integration.keras.callbacks import WandbMetricsLogger
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 00_jw (jiwoong-team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13
current_dir = os.getcwd()
csv_dir = os.path.join(current_dir, 'data', 'raw_csv')
train_csv_path = os.path.join(csv_dir, 'merged_train.csv')

/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13


In [4]:
INPUT_MAX_LENGTH = 350  # 입력 시퀀스 최대 길이
NUM_WORDS = 7000 # 토큰화에 사용할 단어 갯수
EMBEDDING_DIM = 100  # 임베딩 차원(100차원 사전학습 임베딩)

In [5]:
df = pd.read_csv(train_csv_path)
ordered_columns = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화'] # kaggle에 명시된 순서로 재배치

conversations = df['conversation'].to_list() # input
labels = pd.get_dummies(df['class'])
labels = labels[ordered_columns].to_numpy() # target

In [6]:
# 훈련셋,검증셋 분리
X_train_raw, X_val_raw, y_train, y_val = train_test_split(conversations, labels, test_size=0.2, stratify=labels, random_state=42)

In [7]:
# 토크나이저 생성
tokenizer = create_tokenizer(conversations, NUM_WORDS)
print("생성된 어휘 수:", len(tokenizer.word_index))

생성된 어휘 수: 55065


In [8]:
# 전처리 수행
X_train = preprocessing(X_train_raw, tokenizer, INPUT_MAX_LENGTH)
X_val = preprocessing(X_val_raw, tokenizer, INPUT_MAX_LENGTH)

전처리된 데이터는 계속해서 반복되므로 피클파일로 저장

In [9]:
import pickle

# 전처리된 데이터 저장
with open('/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13/notebooks/jiwoong/data/preprocessed_data_no_aug.pkl', 'wb') as f:
    pickle.dump({'X_train': X_train, 'X_val': X_val, 'y_train': y_train, 'y_val': y_val}, f)

# # 불러오기
# with open('preprocessed_data.pkl', 'rb') as f:
#     data = pickle.load(f)
#     X_train = data['X_train']
#     X_val = data['X_val']
#     y_train = data['y_train']
#     y_val = data['y_val']

사전학습 워드 임베딩

In [10]:
# 임베딩 행렬 생성 함수
def build_embedding_matrix(ft_model, tokenizer, num_words, embedding_dim):
    vocab_size = num_words + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    for word, idx in tokenizer.word_index.items():
        if idx >= vocab_size:
            continue
        embedding_vector = ft_model.get_word_vector(word)
        if embedding_vector is not None:
            embedding_matrix[idx] = embedding_vector
    return embedding_matrix

In [13]:
# 사전학습된 임베딩 불러오기
ft = fasttext.load_model('/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13/notebooks/jiwoong/data/cc.ko.100.bin')
embedding_matrix = build_embedding_matrix(ft, tokenizer, NUM_WORDS, EMBEDDING_DIM)

In [14]:
# 워드 임베딩을 파인튜닝 하지 않은 모델
def build_model_pretrained_embedding(embedding_matrix):
    vocab_size = embedding_matrix.shape[0]

    inputs = Input(shape=(INPUT_MAX_LENGTH,))
    embedding = Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM,
                          weights=[embedding_matrix],
                          input_length=INPUT_MAX_LENGTH,
                          trainable=False)(inputs)
    lstm = Bidirectional(LSTM(64))(embedding)
    dropout = Dropout(0.5)(lstm)
    outputs = Dense(5, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=outputs)

    return model

model = build_model_pretrained_embedding(embedding_matrix)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 350)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 350, 100)       │       700,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 785,225 (3.00 MB)

 Trainable params: 85,125 (332.52 KB)

 Non-trainable params: 700,100 (2.67 MB)

In [15]:
# F1Score 평가 지표
f1 = F1Score(average='micro', name='f1')  # micro f1: 전체 클래스 f1score 평균

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1])

es_callback = EarlyStopping(
    monitor='val_f1',
    patience=3,
    restore_best_weights=True,
    mode='max'
)

mc_callback = ModelCheckpoint(
  f'./notebooks/jiwoong/models/model_pretrained_emb.keras',
  monitor="val_f1",
  save_best_only=True,
  mode="max"
)

# W&B 프로젝트 초기화
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="jiwoong-team",
    # Set the wandb project where this run will be logged.
    project="DLThon-DKTC",
    # Track hyperparameters and run metadata.
    name=f"jiwoong_BiLSTM_pretrained_emb", # 실험 이름
    notes="LLM으로 생성한 일반 대화 데이터를 사용한 베이스라인 모델에 워드 임베딩 적용", # 실험에 대한 간단한 설명
    config={ # 세부 구성 내용
        "experiment_name": "BiLSTM_pretrained_emb",
        "general_conversation_type": "LLM에서 생성한 대화 데이터", # 일반 데이터 타입
        "architecture": "BiLSTM(64), Pretrained_emb",
    },
)

history = model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[es_callback, mc_callback, WandbMetricsLogger()] # WandbMetricsLogger를 콜백으로 넘겨주기만 하면 알아서 epoch별 metric을 기록
)

# 최적의 모델로 검증세트 예측
y_pred = model.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

# wandb Table 생성
report = classification_report(y_true_labels, y_pred_labels, target_names=ordered_columns, output_dict=True)
columns = ["class", "precision", "recall", "f1-score", "support"]
data = []

for label, metrics in report.items():
    if isinstance(metrics, dict):  # dict일 때만 values 가져오기
        row = [label] + [metrics.get(col, None) for col in columns[1:]]
        data.append(row)

table = wandb.Table(columns=columns, data=data)

# wandb에 훈련 결과 기록
wandb.log({
    "classification_report_table": table,
    "macro_f1": report["macro avg"]["f1-score"],
    "accuracy": report["accuracy"],
    "threat_f1": report["협박 대화"]["f1-score"],
    "extortion_f1": report["갈취 대화"]["f1-score"],
    "workplace_bullying_f1": report["직장 내 괴롭힘 대화"]["f1-score"],
    "other bullying": report["기타 괴롭힘 대화"]["f1-score"],
    "general_conversation_f1": report["일반 대화"]["f1-score"],
})

# 실험 종료
run.finish()

Epoch 1/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.2991 - f1: 0.2991 - loss: 1.5051 - val_accuracy: 0.5528 - val_f1: 0.5528 - val_loss: 1.0188
Epoch 2/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.5620 - f1: 0.5620 - loss: 1.0313 - val_accuracy: 0.7198 - val_f1: 0.7198 - val_loss: 0.7976
Epoch 3/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.6889 - f1: 0.6889 - loss: 0.8212 - val_accuracy: 0.7381 - val_f1: 0.7381 - val_loss: 0.6752
Epoch 4/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.7536 - f1: 0.7536 - loss: 0.6963 - val_accuracy: 0.7974 - val_f1: 0.7974 - val_loss: 0.5806
Epoch 5/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.7665 - f1: 0.7665 - loss: 0.6693 - val_accuracy: 0.7759 - val_f1: 0.7759 - val_loss: 0.6757
Epoch 6/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.7928 - f1: 0.7928 - loss: 0.6133 - val_accuracy: 0.7899 - val_f1: 0.7899 - val_loss: 0.6155
Epoch 7/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 27

accuracy,▁
epoch/accuracy,▁▄▆▇▇▇▇███
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/f1,▁▄▆▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▃▃▂▂▂▁▁▁
epoch/val_accuracy,▁▅▆▇▇▇██▇█
epoch/val_f1,▁▅▆▇▇▇██▇█
epoch/val_loss,█▅▃▂▃▂▂▁▂▁
extortion_f1,▁
general_conversation_f1,▁


In [16]:
y_pred = model.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

print(confusion_matrix(y_true_labels, y_pred_labels))
print(classification_report(y_true_labels, y_pred_labels, target_names=ordered_columns))

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[122  28  11  16   2]
 [  7 172   9   6   1]
 [  1   7 175  10   1]
 [ 18  38   9 137   0]
 [  0   1   2   1 154]]
              precision    recall  f1-score   support

       협박 대화       0.82      0.68      0.75       179
       갈취 대화       0.70      0.88      0.78       195
 직장 내 괴롭힘 대화       0.85      0.90      0.88       194
   기타 괴롭힘 대화       0.81      0.68      0.74       202
       일반 대화       0.97      0.97      0.97       158

    accuracy                           0.82       928
   macro avg       0.83      0.82      0.82       928
weighted avg       0.82      0.82      0.82       928



In [17]:
# 워드 임베딩을 파인튜닝한 모델
def build_model_pretrained_embedding_finetunning(embedding_matrix):
    vocab_size = embedding_matrix.shape[0]

    inputs = Input(shape=(INPUT_MAX_LENGTH,))
    embedding = Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM,
                          weights=[embedding_matrix],
                          input_length=INPUT_MAX_LENGTH,
                          trainable=True)(inputs)
    lstm = Bidirectional(LSTM(64))(embedding)
    dropout = Dropout(0.5)(lstm)
    outputs = Dense(5, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=outputs)

    return model

model = build_model_pretrained_embedding_finetunning(embedding_matrix)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 350)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 350, 100)       │       700,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 785,225 (3.00 MB)

 Trainable params: 785,225 (3.00 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# F1Score 평가 지표
f1 = F1Score(average='micro', name='f1')  # micro f1: 전체 클래스 f1score 평균

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1])

es_callback = EarlyStopping(
    monitor='val_f1',
    patience=3,
    restore_best_weights=True,
    mode='max'
)

mc_callback = ModelCheckpoint(
  f'./notebooks/jiwoong/models/model_pretrained_emb_fine.keras',
  monitor="val_f1",
  save_best_only=True,
  mode="max"
)

# W&B 프로젝트 초기화
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="jiwoong-team",
    # Set the wandb project where this run will be logged.
    project="DLThon-DKTC",
    # Track hyperparameters and run metadata.
    name=f"jiwoong_BiLSTM_pretrained_emb_fine", # 실험 이름
    notes="LLM으로 생성한 일반 대화 데이터를 사용한 베이스라인 모델에 워드 임베딩 적용(파인튜닝))", # 실험에 대한 간단한 설명
    config={ # 세부 구성 내용
        "experiment_name": "BiLSTM_pretrained_emb_fine",
        "general_conversation_type": "LLM에서 생성한 대화 데이터", # 일반 데이터 타입
        "architecture": "BiLSTM(64), Pretrained_emb_fine",
    },
)

history = model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[es_callback, mc_callback, WandbMetricsLogger()] # WandbMetricsLogger를 콜백으로 넘겨주기만 하면 알아서 epoch별 metric을 기록
)

# 최적의 모델로 검증세트 예측
y_pred = model.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

# wandb Table 생성
report = classification_report(y_true_labels, y_pred_labels, target_names=ordered_columns, output_dict=True)
columns = ["class", "precision", "recall", "f1-score", "support"]
data = []

for label, metrics in report.items():
    if isinstance(metrics, dict):  # dict일 때만 values 가져오기
        row = [label] + [metrics.get(col, None) for col in columns[1:]]
        data.append(row)

table = wandb.Table(columns=columns, data=data)

# wandb에 훈련 결과 기록
wandb.log({
    "classification_report_table": table,
    "macro_f1": report["macro avg"]["f1-score"],
    "accuracy": report["accuracy"],
    "threat_f1": report["협박 대화"]["f1-score"],
    "extortion_f1": report["갈취 대화"]["f1-score"],
    "workplace_bullying_f1": report["직장 내 괴롭힘 대화"]["f1-score"],
    "other bullying": report["기타 괴롭힘 대화"]["f1-score"],
    "general_conversation_f1": report["일반 대화"]["f1-score"],
})

# 실험 종료
run.finish()

Epoch 1/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - accuracy: 0.3320 - f1: 0.3320 - loss: 1.4455 - val_accuracy: 0.6584 - val_f1: 0.6584 - val_loss: 0.8946
Epoch 2/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - accuracy: 0.7467 - f1: 0.7467 - loss: 0.7249 - val_accuracy: 0.8125 - val_f1: 0.8125 - val_loss: 0.5691
Epoch 3/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.8650 - f1: 0.8650 - loss: 0.4515 - val_accuracy: 0.7909 - val_f1: 0.7909 - val_loss: 0.5510
Epoch 4/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.9022 - f1: 0.9022 - loss: 0.3406 - val_accuracy: 0.8556 - val_f1: 0.8556 - val_loss: 0.4374
Epoch 5/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.9433 - f1: 0.9433 - loss: 0.2255 - val_accuracy: 0.8761 - val_f1: 0.8761 - val_loss: 0.4122
Epoch 6/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.9596 - f1: 0.9596 - loss: 0.1650 - val_accuracy: 0.8685 - val_f1: 0.8685 - val_loss: 0.4129
Epoch 7/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 35m

accuracy,▁
epoch/accuracy,▁▅▆▇▇██████
epoch/epoch,▁▂▂▃▄▅▅▆▇▇█
epoch/f1,▁▅▆▇▇██████
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▃▂▂▂▁▁▁▁▁
epoch/val_accuracy,▁▆▅▇███████
epoch/val_f1,▁▆▅▇███████
epoch/val_loss,█▃▃▁▁▁▂▁▂▂▂
extortion_f1,▁
general_conversation_f1,▁


In [19]:
y_pred = model.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

print(confusion_matrix(y_true_labels, y_pred_labels))
print(classification_report(y_true_labels, y_pred_labels, target_names=ordered_columns))

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[146  19   3  11   0]
 [  5 175   8   7   0]
 [  2   1 185   6   0]
 [ 15  21  12 154   0]
 [  0   0   2   0 156]]
              precision    recall  f1-score   support

       협박 대화       0.87      0.82      0.84       179
       갈취 대화       0.81      0.90      0.85       195
 직장 내 괴롭힘 대화       0.88      0.95      0.92       194
   기타 괴롭힘 대화       0.87      0.76      0.81       202
       일반 대화       1.00      0.99      0.99       158

    accuracy                           0.88       928
   macro avg       0.89      0.88      0.88       928
weighted avg       0.88      0.88      0.88       928

